# MLProto Demo

This notebook demos a use case of the MLProto LSTM prototyping tool which we will use to demo different model structures. It is a notebook version of the stock.py example found in the examples module of the MLProto source code [here](https://github.com/CSLukeW/MLProto/blob/master/MLProto/Examples/stock.py). 

## Environment

In [2]:
# stock data garnered from alpha_vantage api
from alpha_vantage.timeseries import TimeSeries

# MLProto imports
from MLProto.MLProto import Proto as proto
from MLProto.MLProto import Data as helper

# other dependencies
import argparse
from matplotlib import pyplot
import pandas
import numpy
from sklearn.preprocessing import MinMaxScaler

## Data Collection

The Alpha Vantage API offers a variety of options for pulling historical financial market data. We will be using daily adjusted stats going back 20 years or to the public debut of the company, whichever is closer to present day.

Below is the function we will use to pull data from Alpha Vantage.

In [ ]:
def daily_adjusted(symbol, key, compact=True):
    """ Returns data frame of queried data
        args:
            symbol -- symbol of desired stock
            key -- user's API key
        compact -- True -> last 100 results
                   False -> all past results
    """

    # create time series
    ts = TimeSeries(key=key, output_format='pandas')

    # take last 100 or complete historical as needed
    if compact:
        data, _ = ts.get_daily_adjusted(symbol=symbol, outputsize='compact')
    else:
        data, _ = ts.get_daily_adjusted(symbol=symbol, outputsize='full')

    return data